# 1. 单局 PK 可视化区域

In [2]:
from gomoku.player import ZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple, GomokuEnv
from gomoku.policy import ZeroPolicy
from gomoku.zero_mcgs import ZeroMCGS

/home/smokingmouse/python/ai/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-21 23:54:33,380	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
import torch

board_size = 9

policy = ZeroPolicy(board_size, num_blocks=2, base_channels=32)
# policy.load_state_dict(torch.load('continue_model/policy_step_940000.pth'))
# policy.load_state_dict(torch.load('../models/gomoku_zero_9_pre5/policy_step_800000.pth'))
policy.load_state_dict(torch.load('../models/gomoku_zero_9_lab_3/policy_step_200000.pth'))
player1 = ZeroMCTSPlayer(policy)
player2 = ZeroMCTSPlayer(policy)

game = GomokuEnv(board_size)

info = play_one_game(player1, player2, game=game,board_size=board_size, render=True, eager=True, itermax=200, MCTS=ZeroMCGS)

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  .  .  .  .  . 
 4 .  .  .  X  .  .  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  .  .  .  .  . 
 4 .  .  .  X  .  O  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  .  .  .  .  .  .  . 
 3 .  .  .  .  X  .  .  .  . 
 4 .  .  .  X  .  O  .  .  . 
 5 .  .  .  .  .  .  .  .  . 
 6 .  .  .  .  .  .  .  .  . 
 7 .  .  .  .  .  .  .  .  . 
 8 .  .  .  .  .  .  .  .  . 

   0  1  2  3  4  5  6  7  8
 0 .  .  .  .  .  .  .  .  . 
 1 .  .  .  .  .  .  .  .  . 
 2 .  .  . 

In [ ]:
import cProfile
import torch
import pstats
import io

# --- 开始性能分析 ---

# 1. 创建一个 Profiler 对象
profiler = cProfile.Profile()

# 2. 启用 Profiler 并运行你的函数
profiler.enable()
policy = ZeroPolicy(board_size=15)
policy.load_state_dict(torch.load('models/gomoku_zero_15_continue/policy_step_6000.pth'))
player1 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)
player2 = ZeroMCTSPlayer(policy, itermax=200, device='cpu', eager=True)

game = GomokuEnv(15)

info = play_one_game(player1, player2, board_size=15, render=True)

profiler.disable()

# 3. 创建一个 IO 流来捕获分析结果
s = io.StringIO()

# 4. 创建 pstats.Stats 对象来格式化和排序结果
#    sort_stats() 的参数是排序依据，'cumulative' 是按累计耗时排序
stats = pstats.Stats(profiler, stream=s).sort_stats('cumulative')

# 5. 打印分析报告
stats.print_stats()

# 6. (可选) 只打印前 10 个最耗时的函数
# stats.print_stats(10)

print(s.getvalue())


# 2. ARENA 

In [3]:
from gomoku.player import arena_parallel
from gomoku.player import ZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

In [9]:
import torch
board_size = 9
policy1 = ZeroPolicy(board_size)
policy2 = ZeroPolicy(board_size)
policy1.load_state_dict(torch.load('../models/gomoku_zero_9_lab_4/policy_step_100000.pth'))
policy2.load_state_dict(torch.load('../models/continue_model/policy_step_940000.pth')) 

r = arena_parallel(
    policy1,
    policy2, 
    board_size=9,
    num_cpus=16,
    games=300,
    itermax=100,
)

Starting parallel arena with 300 games on 16 CPUs...
Arena finished!
Player 1 wins: 159 (53.00%)
Player 2 wins: 123 (41.00%)
Draws: 18 (6.00%)
函数 'arena_parallel' 执行耗时: 38.2782 秒


In [ ]:
from worker import gather_selfplay_games


gather_selfplay_games(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    num_workers=10,
    games_per_worker=10,
    itermax=100,
)

In [ ]:
from player import self_play


_ = self_play(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    itermax=100,
)

In [ ]:


_ = play_one_game(
    player1=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    player2=ZeroMCTSPlayer(ZeroPolicy(board_size=9)),
    board_size=9,
    # render=True,
    game=GomokuEnvSimple(board_size=9),
    itermax=100
)

# 3. 评价棋局

In [2]:
from gomoku.player import ZeroMCTSPlayer, play_one_game, self_play
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy
import torch

/home/smokingmouse/python/ai/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-13 01:04:42,711	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
board_size = 15

policy = ZeroPolicy(board_size, 4, 64)
policy.load_state_dict(torch.load('models/gomoku_zero_15_lab_3/policy_step_990000.pth'))

device='cpu'
player1 = ZeroMCTSPlayer(policy, device=device) 
player2 = ZeroMCTSPlayer(policy, device=device)

env = GomokuEnvSimple(board_size)

winner, infos = play_one_game(
    player1, player2, board_size=board_size, game=env,
    itermax=200, eager=True, use_rs_mcts=True
)
# print(f"Game over! Winner: {winner}")
# return infos

函数 'play_one_game' 执行耗时: 3.9222 秒


In [14]:
# infos = self_play(policy, 'cpu', board_size=board_size, itermax=400)

In [15]:
def render(states, index):
    import numpy as np
    states_tensor = torch.from_numpy(np.array(infos['states'], dtype=np.float32))
    probs, winrate = policy(states_tensor)

    symbols = {0: '.', 1: 'X', 2: 'O'}
    board_str = "  " + " ".join([f"{i:2d}" for i in range(board_size)]) + "\n"

    for i in range(board_size):
        cells = []
        for j in range(board_size):
            if states[index][ 0, i, j] ==  1:  
                cell = 1
            elif states[index][ 1, i, j] == 1:
                cell = 2
            else:
                cell = 0
            cells.append(cell)
        board_str += f"{i:2d} " + " ".join([f"{symbols[cell]} " for cell in cells]) + "\n"
    print(board_str)
    print(winrate[index])
    values, indices = torch.topk(torch.softmax(probs, dim=-1)[index], 10)

    def action_2_index(action):
        return action // board_size, action % board_size

    for i  in range(len(indices)):
        a, b = action_2_index(indices[i].item())
        print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


In [16]:
import numpy as np
obs = env._get_observation()
print("last_action", env.last_action, "move_size", env.move_size)
print("obs sum", obs.sum(), "last_action sum", obs[2].sum())
print("last_action ones", np.argwhere(obs[2] == 1))

last_action 110 move_size 24
obs sum 25 last_action sum 1
last_action ones [[7 5]]


In [43]:
render(infos['states'], 24)

IndexError: list index out of range

In [ ]:
render(info[1]['states'], -2)

In [ ]:
def action_2_index(action):
    return action // 9, action % 9

probs = torch.tensor(info[1]['probs'])
values, indices = probs[-2].topk(10)

for i  in range(len(indices)):
    a, b = action_2_index(indices[i].item())
    print(f"[{a:2d},{b:2d}]:{values[i].item():.4f}")


# 4. debug

In [ ]:
from gomoku.player import make_policy_fn
from gomoku.gomoku_env import GomokuEnv
from zero_mcts_rs import LightZeroMCTS
import numpy as np

env = GomokuEnv(board_size=9)
policy = ZeroPolicy(board_size)

def uniform_policy_fn(obs: np.ndarray):
    board_size = obs.shape[-1]
    total = board_size * board_size 
    logits = np.zeros((total, ), dtype=np.float32)    
    return logits, 0.0


policy_fn1 = make_policy_fn(policy, player1.device)
policy_fn2 = make_policy_fn(policy, player2.device)
mcts1 = LightZeroMCTS(
    board_size=board_size,
    puct=2.0,
    dirichlet_alpha=0.3,
    dirichlet_epsilon=0.25,
)
mcts2 = LightZeroMCTS(
    board_size=board_size,
    puct=2.0,
    dirichlet_alpha=0.3,
    dirichlet_epsilon=0.25,
)

In [17]:
current_state = env._get_observation()
board_flat = env.board.astype(np.int8).reshape(-1)

eager = False

mcts1.run(
    board_flat,
    env.current_player,
    policy_fn1,
    iterations=800,
    use_dirichlet=True,
    move_size=env.move_size,
    last_action=env.last_action,
)

num_moves = env.move_size
temperature = (
    0.0 if eager else (1.0 if num_moves < 10 else 0.0)
)
action, probs_for_training = mcts1.select_action_with_temperature(
    temperature, None, None
)

In [18]:
action

50

In [73]:
# --- MCTS sanity: uniform prior (no dirichlet, temperature=0) ---
from zero_mcts_rs import LightZeroMCTS
import numpy as np
from gomoku.gomoku_env import GomokuEnv

board_size = 9
env = GomokuEnv(board_size=board_size)


def uniform_policy_fn(obs: np.ndarray):
    board_size = obs.shape[-1]
    total = board_size * board_size
    logits = np.zeros((total,), dtype=np.float32)
    return logits, 0.0

mcts = LightZeroMCTS(
    board_size=board_size,
    puct=2.0,
    dirichlet_alpha=0.3,
    dirichlet_epsilon=0.25,
)
board_flat = env.board.astype(np.int8).reshape(-1)
mcts.run(
    board_flat,
    env.current_player,
    uniform_policy_fn,
    iterations=800,
    use_dirichlet=False,
    move_size=env.move_size,
    last_action=env.last_action,
)
action, pi = mcts.select_action_with_temperature(temperature=0, top_k=None)

pi = np.array(pi)
idx = np.argsort(pi)[::-1]
print("Top-10 pi (action, row, col, prob):")
for i in idx[:10]:
    r, c = divmod(int(i), board_size)
    print(i, r, c, float(pi[i]))
print("Selected action:", action, divmod(int(action), board_size))


Top-10 pi (action, row, col, prob):
80 8 8 0.012500000186264515
79 8 7 0.012500000186264515
78 8 6 0.012500000186264515
77 8 5 0.012500000186264515
75 8 3 0.012500000186264515
73 8 1 0.012500000186264515
74 8 2 0.012500000186264515
72 8 0 0.012500000186264515
71 7 8 0.012500000186264515
67 7 4 0.012500000186264515
Selected action: 73 (8, 1)


In [74]:
# --- MCTS sanity: center-biased prior (should pick center) ---
from zero_mcts_rs import LightZeroMCTS
import numpy as np
from gomoku.gomoku_env import GomokuEnv

board_size = 9
env = GomokuEnv(board_size=board_size)


def center_biased_policy_fn(obs: np.ndarray):
    board_size = obs.shape[-1]
    total = board_size * board_size
    logits = np.zeros((total,), dtype=np.float32)
    center = (board_size // 2) * board_size + (board_size // 2)
    logits[center] = 6.0
    return logits, 0.0

mcts = LightZeroMCTS(
    board_size=board_size,
    puct=2.0,
    dirichlet_alpha=0.3,
    dirichlet_epsilon=0.25,
)
board_flat = env.board.astype(np.int8).reshape(-1)
mcts.run(
    board_flat,
    env.current_player,
    center_biased_policy_fn,
    iterations=400,
    use_dirichlet=False,
    move_size=env.move_size,
    last_action=env.last_action,
)
action, pi = mcts.select_action_with_temperature(temperature=0, top_k=None)

pi = np.array(pi)
idx = np.argsort(pi)[::-1]
print("Top-10 pi (action, row, col, prob):")
for i in idx[:10]:
    r, c = divmod(int(i), board_size)
    print(i, r, c, float(pi[i]))
print("Selected action:", action, divmod(int(action), board_size))


Top-10 pi (action, row, col, prob):
40 4 4 0.9975000023841858
78 8 6 0.0024999999441206455
80 8 8 0.0
77 8 5 0.0
79 8 7 0.0
75 8 3 0.0
74 8 2 0.0
73 8 1 0.0
76 8 4 0.0
71 7 8 0.0
Selected action: 40 (4, 4)


In [75]:
from gomoku.player import self_play

In [77]:
self_play(
    policy=ZeroPolicy(board_size=9),
    device='cpu',
    board_size=9,
    itermax=200,
    use_rs_mcts=True,
    branch_ratio=1.0
)

函数 'play_one_game' 执行耗时: 1.8784 秒
函数 'play_one_game' 执行耗时: 1.0253 秒
Branch game over! Winner: 2
Game over! Winner: 2


[{'states': [array([[[0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0]],
   
          [[0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0]],
   
          [[0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 0, 0,

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from gomoku.policy import ZeroPolicy  # 替换成你的 Policy 文件路径

def test_overfit():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    board_size = 9
    
    # 1. 构造“唯一”的样本
    # 状态：空棋盘 (3, 9, 9) -> Channel 0, 1 都是全0, Channel 2 (Last Move) 全0
    # 注意：这里模拟 Rust/Env 的输出格式，全是 0.0/1.0
    fake_state = np.zeros((3, 9, 9), dtype=np.float32) 
    
    # 标签：只允许走天元 (Action = 4*9 + 4 = 40)
    target_action = 40 
    target_policy = np.zeros(81, dtype=np.float32)
    target_policy[target_action] = 1.0
    
    # 价值：黑棋必胜 (+1)
    target_value = np.array([1.0], dtype=np.float32)

    # 2. 初始化网络
    model = ZeroPolicy(board_size=9).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01) # 大火收汁

    print("🚀 开始过拟合测试：目标是让 AI 学会第一手必走天元")
    
    # 3. 疯狂训练这一个样本 1000 次
    model.train()
    inputs = torch.from_numpy(fake_state).unsqueeze(0).float().to(device) # (1, 3, 9, 9)
    target_p = torch.from_numpy(target_policy).unsqueeze(0).to(device)    # (1, 81)
    target_v = torch.from_numpy(target_value).unsqueeze(0).to(device)     # (1, 1)

    for i in range(500):
        optimizer.zero_grad()
        p_logits, v_out = model(inputs)
        
        # Loss 计算
        loss_v = nn.MSELoss()(v_out, target_v)
        loss_p = -torch.sum(target_p * torch.nn.functional.log_softmax(p_logits, dim=1))
        
        loss = 2 * loss_v + loss_p
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            probs = torch.softmax(p_logits, dim=1).detach().cpu().numpy()[0]
            center_prob = probs[target_action]
            val = v_out.item()
            print(f"Step {i}: Loss={loss.item():.4f}, Center_Prob={center_prob:.4f}, Value={val:.4f}")

    # 4. 最终验证
    print("\n🔍 最终结果检查:")
    p_logits, _ = model(inputs)
    probs = torch.softmax(p_logits, dim=1).detach().cpu().numpy()[0]
    best_move = np.argmax(probs)
    print(f"AI 认为最好的一步: {best_move} (坐标 {best_move//9}, {best_move%9})")
    print(f"天元概率: {probs[40]:.4f}")
    
    if best_move == 40 and probs[40] > 0.9:
        print("✅ 通过测试：网络结构和梯度传递是正常的！")
    else:
        print("❌ 测试失败：网络连这一个样本都学不会！问题出在 ZeroPolicy 的 forward 里！")

if __name__ == "__main__":
    test_overfit()

🚀 开始过拟合测试：目标是让 AI 学会第一手必走天元
Step 0: Loss=6.4154, Center_Prob=0.0123, Value=-0.0048
Step 100: Loss=0.0000, Center_Prob=1.0000, Value=1.0000
Step 200: Loss=0.0000, Center_Prob=1.0000, Value=1.0000
Step 300: Loss=0.0000, Center_Prob=1.0000, Value=1.0000
Step 400: Loss=0.0000, Center_Prob=1.0000, Value=1.0000

🔍 最终结果检查:
AI 认为最好的一步: 40 (坐标 4, 4)
天元概率: 1.0000
✅ 通过测试：网络结构和梯度传递是正常的！


In [54]:
from gomoku.gomoku_env import GomokuEnv
env = GomokuEnv(9)
env.step(40) # 黑棋走天元

obs = env._get_observation() 
# obs shape (3, 9, 9)
# Channel 0: P2 (White) -> 应全0
# Channel 1: P1 (Black) -> 应在 (4,4) 有值
# Channel 2: Last Move  -> 应在 (4,4) 有值

print("Channel 0 (P2) Sum:", obs[0].sum()) # 应该是 0
print("Channel 1 (P1) Sum:", obs[1].sum()) # 应该是 1
print("Channel 2 (Last) Sum:", obs[2].sum()) # 应该是 1

# 关键检查：打印非零位置的坐标
import numpy as np
y, x = np.where(obs[1] == 1)
print(f"黑棋位置: ({y[0]}, {x[0]})") # 必须是 (4, 4)！

# 如果这里输出的是 (4, 4)，那么再检查 Rust 端的数据
# (这一步比较难，因为涉及到 Rust debug，暂且假设 Python 端是对的)

Channel 0 (P2) Sum: 0
Channel 1 (P1) Sum: 1
Channel 2 (Last) Sum: 1
黑棋位置: (4, 4)


In [1]:
from gomoku.gomoku_env import GomokuEnv

env = GomokuEnv()

In [1]:
import time

start = time.time()
b = env.clone()
print("Elapsed:", time.time() - start)

NameError: name 'env' is not defined

In [2]:
from gomoku.player import self_play
from gomoku.policy import ZeroPolicy

/home/smokingmouse/python/ai/GomokuZero/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-14 23:11:34,010	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
self_play(
    policy=ZeroPolicy(board_size=15),
    device='cpu',
    board_size=15,
    itermax=400,
)

In [4]:
model = ZeroPolicy(15) 

In [7]:
import torchsummary

In [10]:
torchsummary.summary(model, (3, 15, 15), 256, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [256, 64, 15, 15]           1,792
       BatchNorm2d-2          [256, 64, 15, 15]             128
            Conv2d-3          [256, 64, 15, 15]          36,928
       BatchNorm2d-4          [256, 64, 15, 15]             128
            Conv2d-5          [256, 64, 15, 15]          36,928
       BatchNorm2d-6          [256, 64, 15, 15]             128
          ResBlock-7          [256, 64, 15, 15]               0
            Conv2d-8          [256, 64, 15, 15]          36,928
       BatchNorm2d-9          [256, 64, 15, 15]             128
           Conv2d-10          [256, 64, 15, 15]          36,928
      BatchNorm2d-11          [256, 64, 15, 15]             128
         ResBlock-12          [256, 64, 15, 15]               0
           Conv2d-13          [256, 64, 15, 15]          36,928
      BatchNorm2d-14          [256, 64,